# Project 15: Analysis - Comparing Pretrained vs Random Model

## Goal
Rigorously compare pretrained model to untrained baseline.

## Learning Objectives
- Value of pretraining
- Transfer learning effectiveness
- Why fine-tuning beats training from scratch

In [ ]:
# Setup
import torch
import numpy as np
import matplotlib.pyplot as plt

print("Ready to analyze pretrained models!")